# AISOC Workshop: Feature Engineering and Selection

NOTE: This notebook is currently in progress

## 1. Techniques for Feature Engineering

- Common operations for data preprocessing - drop columns, missing values, date conversion , etc

- Feature transformation - scaling, log transform, Binning & Discretization

- Feature extraction 
    - Time-based features: month, quarter, weekday, seasonality
    - Customer/delivery-related features: shipping region, delivery status.
    - Ratio features (Amount per Qty, etc.).
    - Promotional features (binary flag for promotion presence).

- Encoding techniques 
    - One-hot encoding (Category, Fulfilment, etc.), label encoding, target encoding, 
    - Frequency encoding for high-cardinality variables (SKU, Style, ASIN, ship-city)

- Interaction & Polynomial Features
    - Cross features (Category × Size)
    - Multiplicative features (Qty × Avg Price)
    - Polynomial features


### Common operations for data preprocessing

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("Amazon Sale Report.csv")

C:\Users\pibzion\AppData\Local\Temp\ipykernel_59216\1208789707.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\pibzion\Downloads\Amazon Sale Report.csv")


In [4]:
df.head(3)

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

***Drop unwanted columns***

In [6]:
cols_to_drop = ["index", "Unnamed: 22"]
df_clean = df.drop(columns=cols_to_drop)

In [7]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Order ID            128975 non-null  object 
 1   Date                128975 non-null  object 
 2   Status              128975 non-null  object 
 3   Fulfilment          128975 non-null  object 
 4   Sales Channel       128975 non-null  object 
 5   ship-service-level  128975 non-null  object 
 6   Style               128975 non-null  object 
 7   SKU                 128975 non-null  object 
 8   Category            128975 non-null  object 
 9   Size                128975 non-null  object 
 10  ASIN                128975 non-null  object 
 11  Courier Status      122103 non-null  object 
 12  Qty                 128975 non-null  int64  
 13  currency            121180 non-null  object 
 14  Amount              121180 non-null  float64
 15  ship-city           128942 non-nul

### Feature transformation techniques

***Scaling***

Min-max: Uses the minimun and maximum values of a feature to rescale values to within a range.

$$x' = \frac{x_i-min(x)}{max(x)-min(x)}$$

Standard scaling: Scales the data such that it has a mean of 0 and a standard deviation of 1.

$$x' = \frac{x_i-\bar{x}}{\sigma}$$

Robust scaling: Improves on standard scaling to avoid negative impact of outliers, by using the median (Q2) and the interquartile range (Q3-Q1).

<br>

**PS: Interquartile range**

$$x' = \frac{x_i-Q2}{IQR}$$

- It is a measure of the spread of the middle 50% of a dataset. 
- It represents the difference between the 1st quartile (Q1) and 3rd quartile (Q3) where median is the 2nd quartile (Q2)
- Q1 is the median of the lower half of the data, while Q3 is the median of the upper half



In [20]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [ ]:
def check_feature_range(col):
    return (min(col), max(col))

num_cols = ["Qty", "Amount"]

for col in num_cols:
    feat = df[col]
    print(check_feature_range(feat))

(0, 15)
(0.0, 5584.0)


In [ ]:
df_scaled = df_clean.copy()

In [39]:
minmax_scaler = MinMaxScaler()
std_scaler = StandardScaler()
robust_scaler = RobustScaler()

In [ ]:
minmax_cols = [col+"_minmax" for col in num_cols]

df_scaled[minmax_cols] = minmax_scaler.fit_transform(df_clean[num_cols])
df_scaled.head(3)

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Qty_minmax,Amount_minmax
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,...,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,0.000000,0.115978
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,0.066667,0.072708
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,0.066667,0.058918


In [21]:
std_cols = [col+"_std" for col in num_cols]
df_scaled[std_cols] = std_scaler.fit_transform(df_clean[num_cols])
df_scaled.head(3)

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Qty_minmax,Amount_minmax,Qty_std,Amount_std
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,...,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,0.000000,0.115978,-2.886307,-0.003348
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,0.066667,0.072708,0.304989,-0.862562
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,0.066667,0.058918,0.304989,-1.136378


In [40]:
robust_cols = [col+"_robust" for col in num_cols]
df_scaled[robust_cols] = robust_scaler.fit_transform(df_clean[num_cols])
df_scaled.head(3)

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,ship-country,promotion-ids,B2B,fulfilled-by,Qty_minmax,Amount_minmax,Qty_std,Amount_std,Qty_robust,Amount_robust
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,...,IN,NaN,False,Easy Ship,0.000000,0.115978,-2.886307,-0.003348,-1.0,0.125723
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,0.066667,0.072708,0.304989,-0.862562,0.0,-0.587021
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,0.066667,0.058918,0.304989,-1.136378,0.0,-0.814159


In [ ]:
for col in minmax_cols:
    feat = df_scaled[col]
    print(check_feature_range(feat))

(0.0, 1.0)
(0.0, 1.0)


In [22]:
for col in std_cols:
    feat = df_scaled[col]
    print(check_feature_range(feat))

(-2.8863068398180336, 44.98312513116297)
(-2.306320008230719, 17.550689119906774)


In [42]:
for col in robust_cols:
    feat = df_scaled[col]
    print(check_feature_range(feat))

(-1.0, 14.0)
(-1.7846607669616519, 14.687315634218288)


### Feature extraction techniques

**Time-based features**

Extract `day`, `month`, `year`, `day_of_week`, `is_weekend` (bool) - these are useful for seasonality, trends, behavior analysis

In [26]:
df["Date"].dtype

dtype('O')

In [31]:
df_temporal = df_clean.copy()

df_temporal["Date"] = pd.to_datetime(df_temporal["Date"], format="%m-%d-%y")
df_temporal["Date"]

0        2022-04-30
1        2022-04-30
2        2022-04-30
3        2022-04-30
4        2022-04-30
            ...    
128970   2022-05-31
128971   2022-05-31
128972   2022-05-31
128973   2022-05-31
128974   2022-05-31
Name: Date, Length: 128975, dtype: datetime64[ns]

In [35]:
df_temporal["Date"].dtype

dtype('<M8[ns]')

In [36]:
df_temporal["order_day"] = df_temporal["Date"].dt.day
df_temporal["order_month"] = df_temporal["Date"].dt.month
df_temporal["order_year"] = df_temporal["Date"].dt.year
df_temporal["order_dayofweek"] = df_temporal["Date"].dt.dayofweek
df_temporal["is_weekend"] = df_temporal["order_dayofweek"].isin([5,6]).astype(int)

In [37]:
df_temporal.head(3)

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,fulfilled-by,Qty_minmax,Amount_minmax,Qty_std,Amount_std,order_day,order_month,order_year,order_dayofweek,is_weekend
0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,...,Easy Ship,0.000000,0.115978,-2.886307,-0.003348,30,4,2022,5,1
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,Easy Ship,0.066667,0.072708,0.304989,-0.862562,30,4,2022,5,1
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,...,NaN,0.066667,0.058918,0.304989,-1.136378,30,4,2022,5,1


In [38]:
df_temporal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Order ID            128975 non-null  object        
 1   Date                128975 non-null  datetime64[ns]
 2   Status              128975 non-null  object        
 3   Fulfilment          128975 non-null  object        
 4   Sales Channel       128975 non-null  object        
 5   ship-service-level  128975 non-null  object        
 6   Style               128975 non-null  object        
 7   SKU                 128975 non-null  object        
 8   Category            128975 non-null  object        
 9   Size                128975 non-null  object        
 10  ASIN                128975 non-null  object        
 11  Courier Status      122103 non-null  object        
 12  Qty                 128975 non-null  int64         
 13  currency            121180 no